In [1]:
from vizdoom import *
import random
import time
import numpy as np
!cd github

In [8]:
game = DoomGame()
game.load_config('github/VizDoom/scenarios/defend_the_center.cfg')
game.init()

In [9]:
actions = np.identity(3,dtype = np.uint8)

In [29]:
random.choice(actions)

array([0, 1, 0], dtype=uint8)

In [30]:
game.new_episode()

In [31]:
game.is_episode_finished

<bound method PyCapsule.is_episode_finished of <vizdoom.vizdoom.DoomGame object at 0x000002186FF64970>>

In [124]:
game.make_action(random.choice(actions))

-100.0

In [125]:
# Loop through episodes 
episodes = 10 
for episode in range(episodes): 
    # Create a new episode or game 
    game.new_episode()
    # Check the game isn't done 
    while not game.is_episode_finished(): 
        # Get the game state 
        state = game.get_state()
        # Get the game image from game state
        img = state.screen_buffer
        # Get the game variables - ammo from state
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)#4 over here is for frame buffer we skip next 4 frame after actions to see the reward of the action
        # Print rewward 
        print('reward:', reward) 
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)

reward: 0.0
reward: 0.0
reward: 0.0
reward: -6.6373291015625
reward: -9.162063598632812
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -0.0898895263671875
reward: -100.0
Result: -115.8892822265625
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -15.819534301757812
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -100.0
Result: -115.81953430175781
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -15.815673828125
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -100.0
Result: -115.815673828125
reward: 0.0
reward: 0.0
reward: 0.0
reward: -5.5651702880859375
reward: -8.164230346679688
reward: -2.1905517578125
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
r

In [126]:
game.close()

In [127]:
from gym import Env
from gym.spaces import Discrete, Box
import cv2

In [128]:
# Create Vizdoom OpenAI Gym Environment
class VizDoomGym(Env): 
    # Function that is called when we start the env
    def __init__(self, render=False): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = DoomGame()
        self.game.load_config('github/VizDoom/scenarios/defend_the_center.cfg')
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(3)
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4) 
        
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [148]:
env = VizDoomGym(render=True) 

In [130]:
from stable_baselines3.common import env_checker
env_checker.check_env(env)

In [183]:
env.step(2)

(array([[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        ...,
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]]),
 -100.0,
 True,
 {'info': 0})

In [184]:
env.close()

In [185]:
import os 
from stable_baselines3.common.callbacks import BaseCallback

In [186]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [187]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [188]:
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)

In [189]:
from stable_baselines3 import PPO

In [190]:
env = VizDoomGym()

In [ ]:
model = PPO('CnnPolicy', env, verbose=1, learning_rate=0.0001, n_steps=2048)